# 1. Setup project

### Load and Visualize AOI from a Local Shapefile (GeoPandas + Folium + Earth Engine)

This section of the notebook performs the following steps:

1. **Check for the presence of shapefiles** in the local directory (`../data/aoi/`).
2. **Authenticate and initialize Google Earth Engine (GEE)** to enable access to remote sensing datasets.
3. **Dynamically import a helper script** (`aoi_loader.py`) containing utility functions to load shapefiles as:

   * a `GeoDataFrame` (for local use and visualization),
   * and an `ee.Geometry` object (for Earth Engine processing).
4. **Load the Area of Interest (AOI)** from the shapefile as both:

   * a `GeoDataFrame` (`aoi_gdf`) using GeoPandas, and
   * an Earth Engine geometry (`aoi_ee`) for cloud-based geospatial analysis.
5. **Create an interactive map** using `folium`, centered on the AOI, and add the AOI layer to it.
6. **Display the map directly within the notebook** (and optionally save it as an HTML file for external viewing).

> 💡 This setup allows smooth interoperability between local GIS data and cloud processing via Earth Engine.

---

> ** Note: Earth Engine Authentication**
>
> When you run `ee.Authenticate()` for the first time, a browser window opens asking you to log in to your Google account and authorize Earth Engine access.
> Once authorized, a **credentials token is saved locally** so that you don’t have to log in again every time.
>
> **Where is the token stored?**
>
> * **Windows:**
>   `C:\Users\<YourUsername>\.config\earthengine\credentials`
>
> * **Linux/macOS:**
>   `~/.config/earthengine/credentials`
>
> On subsequent runs, Earth Engine loads the saved credentials automatically.
>
> **To force re-authentication**, you can:
>
> * Delete or rename the `credentials` file manually, or
> * Use:
>
>   ```python
>   ee.Authenticate(clear_credentials=True)
>   ```

In [ ]:
import os
import ee

print(os.path.exists("../data/aoi/aoi_4326.shp"))

ee.Authenticate()
ee.Initialize()

True


In [ ]:
import importlib.util
import os
import sys

def load_module(name, rel_path):
    module_path = os.path.abspath(rel_path)
    spec = importlib.util.spec_from_file_location(name, module_path)
    module = importlib.util.module_from_spec(spec)
    sys.modules[name] = module
    spec.loader.exec_module(module)
    return module

sys.path.append(os.path.abspath("../scripts"))

aoi_loader      = load_module("aoi_loader", "../scripts/aoi_loader.py")
download_s1     = load_module("download_s1", "../scripts/download_s1.py")
conv_db_sm     = load_module("download_s1", "../scripts/conv_db_sm.py")




In [ ]:
# 1. load AOI shapefile
print(os.path.exists("../data/aoi/aoi_1km7x14_3035.shp"))

# Earth Engine AOI
aoi_ee = aoi_loader.load_shapefile_as_ee("../data/aoi/aoi_1km7x14_3035.shp")

# GeoDataFrame AOI (folium)
aoi_gdf = aoi_loader.load_shapefile_as_gdf("../data/aoi/aoi_1km7x14_3035.shp")
print(type(aoi_gdf))

# map view folium
import folium
m = folium.Map(location=center_coords, zoom_start=12)
folium.GeoJson(aoi_gdf).add_to(m)

m.save("map_aoi.html")
print("💾 Saved 'map_aoi.html' in the current directory")

from IPython.display import display
display(m)

True
<class 'geopandas.geodataframe.GeoDataFrame'>
💾 Saved 'map_aoi.html' in the current directory


In [ ]:
img = get_s1_vv_image(aoi_ee, "2024-04-01", "2024-09-30")

import geemap.foliumap as geemap

Map = geemap.Map(center=[center_coords[0], center_coords[1]], zoom=12)
Map.addLayer(img, {"min": -25, "max": 0}, "SAR VV (Apr–Sep)")
Map.addLayer(ee.FeatureCollection(aoi_ee), {}, "AOI")
Map


In [ ]:
img = get_s1_vv_image(aoi_ee, "2024-04-01", "2024-09-30")

img_clipped = img.clip(aoi_ee)

import geemap.foliumap as geemap

Map = geemap.Map(center=center_coords, zoom=12)
Map.addLayer(img_clipped, {"min": -25, "max": 0}, "SAR VV Clipped")
Map.addLayer(ee.FeatureCollection(aoi_ee), {}, "AOI")
Map


In [ ]:
import ee

task = ee.batch.Export.image.toDrive(
    image=img_clipped,
    description='export_sar_vv_clipped',
    folder='earthengine',  # 📁 Google Drive folder
    fileNamePrefix='sarvv2024_04_09',
    scale=10,  # 10m
    region=aoi_ee,
    fileFormat='GeoTIFF'
)

task.start()
print("🚀 Export in GoogleDrive complite: https://code.earthengine.google.com/tasks ")


🚀 Export avviato: controlla su https://code.earthengine.google.com/tasks per lo stato
